## Surya Teja Pidakala - U99286608

# Average Price Prediction 

#### We have a set of 16 Predictors that can help us find if a person has diabetes or not. Those Predictors are "AGE, GENDER, POLYURIA, POLYDIPSIA, SUDDEN_WEIGHT_LOSS, WEAKNESS, POLYPHAGIA, GENITAL_THRUSH, VISUAL_BLURRING, ITCHING, IRRITABILITY, DELAYED_HEALING, PARTIAL_PARESIS, MUSCLE_STIFFNESS, ALOPECIA and OBESITY"

Before anything, let us discuss the Confusion Matrix in detail.

* True Positive  - This is the case when we predict a person has diabetes and they actually does have diabetes. This a very perfect situation for us because we were right in assessing the situation. So, from a price point, We would have some $10000

* True Negative  - This is the case when we predict a person does not have diabetes and he infact doesn't. This is a case where when we spend nothing and it costs us nothing techinically $0

* False Positive - This is a case when we predict a person has diabetes but he is healthy. Here, we are wrong in assesing the situation and we willl leave the patient with some unnecessary tests and trauma. Let us put this are $2000.

* False Negative - This is the case when we predict a person is healthy but he has diabetes. This a very bad failure for us because this can lead us somewhere very bad. It can cost us the person's life. So let us put this at $100,000.

Finally, we can see that False Negatives are something that we should avoid as much as we can

# 1.0 Let us import all the necessary libraries first!

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score ,f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.utils import resample
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### 1.1 Let us now load the data into a variable

In [35]:
df = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/Online_website.csv")
df.head(5)

,year,month,day,order,country,session ID,page 1 (main category),page 2 (clothing model),colour,location,model photography,price,Avg Price,page
0,2008,4,1,1,29,1,1,A13,1,5,1,28,0,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,0,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,0,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


Let us know the type of data that we are dealing with

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165474 entries, 0 to 165473
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   year                     165474 non-null  int64 
 1   month                    165474 non-null  int64 
 2   day                      165474 non-null  int64 
 3   order                    165474 non-null  int64 
 4   country                  165474 non-null  int64 
 5   session ID               165474 non-null  int64 
 6   page 1 (main category)   165474 non-null  int64 
 7   page 2 (clothing model)  165474 non-null  object
 8   colour                   165474 non-null  int64 
 9   location                 165474 non-null  int64 
 10  model photography        165474 non-null  int64 
 11  price                    165474 non-null  int64 
 12  Avg Price                165474 non-null  int64 
 13  page                     165474 non-null  int64 
dtypes: int64(13), object

To make the data clean, let us convert all words to upper case and replace space with an "_"

In [37]:
df.columns = [s.strip().upper().replace(' ', '_') for s in df.columns] 

In [38]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'ORDER', 'COUNTRY', 'SESSION_ID',
       'PAGE_1_(MAIN_CATEGORY)', 'PAGE_2_(CLOTHING_MODEL)', 'COLOUR',
       'LOCATION', 'MODEL_PHOTOGRAPHY', 'PRICE', 'AVG_PRICE', 'PAGE'],
      dtype='object')

In [39]:
df.head()

,YEAR,MONTH,DAY,ORDER,COUNTRY,SESSION_ID,PAGE_1_(MAIN_CATEGORY),PAGE_2_(CLOTHING_MODEL),COLOUR,LOCATION,MODEL_PHOTOGRAPHY,PRICE,AVG_PRICE,PAGE
0,2008,4,1,1,29,1,1,A13,1,5,1,28,0,1
1,2008,4,1,2,29,1,1,A16,1,6,1,33,0,1
2,2008,4,1,3,29,1,2,B4,10,2,1,52,1,1
3,2008,4,1,4,29,1,2,B17,6,6,2,38,0,1
4,2008,4,1,5,29,1,2,B8,4,3,2,52,1,1


Now, let us see what are the columns that do not contribute towards influencing the target variable. Such columns are of no use to include in our analysis. So, let us go ahead and drop them.

In [40]:
df = df.drop(columns= ['PAGE_1_(MAIN_CATEGORY)', 'PAGE_2_(CLOTHING_MODEL)', 'COLOUR', 'MODEL_PHOTOGRAPHY','SESSION_ID'])

In [41]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'ORDER', 'COUNTRY', 'LOCATION', 'PRICE',
       'AVG_PRICE', 'PAGE'],
      dtype='object')

Let us check for null values in the data after dropping the columns.

In [42]:
df[df.isnull().any(axis=1)]

,YEAR,MONTH,DAY,ORDER,COUNTRY,LOCATION,PRICE,AVG_PRICE,PAGE


We can see there are no Null values in the data. So we can go ahead to Encoding

We can now see that there are lot of categorical values in lot of columns. To make our jobs easy, lets encode them.

In [28]:
# df = df.join(pd.get_dummies(df['YEAR'], prefix='Year', drop_first=True))
# df.drop('YEAR', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['MONTH'], prefix='Month', drop_first=True))
# df.drop('MONTH', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['DAY'], prefix='Day', drop_first=True))
# df.drop('DAY', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['ORDER'], prefix='Order', drop_first=True))
# df.drop('ORDER', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['COUNTRY'], prefix='Country', drop_first=True))
# df.drop('COUNTRY', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['LOCATION'], prefix='Loc', drop_first=True))
# df.drop('LOCATION', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['PRICE'], prefix='Price', drop_first=True))
# df.drop('PRICE', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['AVG_PRICE'], prefix='Avg_price', drop_first=True))
# df.drop('AVG_PRICE', axis=1, inplace = True)

# df = df.join(pd.get_dummies(df['PAGE'], prefix='Page', drop_first=True))
# df.drop('PAGE', axis=1, inplace = True)

In [43]:
df.head(20)

,YEAR,MONTH,DAY,ORDER,COUNTRY,LOCATION,PRICE,AVG_PRICE,PAGE
0,2008,4,1,1,29,5,28,0,1
1,2008,4,1,2,29,6,33,0,1
2,2008,4,1,3,29,2,52,1,1
3,2008,4,1,4,29,6,38,0,1
4,2008,4,1,5,29,3,52,1,1
5,2008,4,1,6,29,1,57,1,4
6,2008,4,1,7,29,1,33,0,4
7,2008,4,1,8,29,5,38,1,4
8,2008,4,1,9,29,4,48,1,5
9,2008,4,1,1,29,5,57,1,2


## 2.0 Predictors and Target Variable split

### 2.1 Partitioning data into training and test data sets


In [44]:
df_train, df_test = train_test_split(df, test_size=0.3)

## 2.2 Let us now divide the set of variables into Target and Predictor classes.

In [48]:
target = 'AVG_PRICE'
predictors = list(df.columns)
predictors.remove(target)

In [49]:
train_X = df_train[predictors]
test_X = df_test[predictors]
train_y = df_train[target]
test_y = df_test[target]

Let us know some key aspects with regards to the balance of data.

We can see that there is an slight imbalance in the data, So lets address this issue

## 2.3 Let us use a Random OverSampler to resize the data

In [50]:
train_y.value_counts()

1    59463
0    56368
Name: AVG_PRICE, dtype: int64

In [52]:
ros = RandomOverSampler(random_state=1)
train_X, train_y = ros.fit_resample(train_X, train_y)
train_y.value_counts()

1    59463
0    59463
Name: AVG_PRICE, dtype: int64

## 2.4 Now, as we addressed the data imbalance issue, we can go ahead with the analysis of the data

## 2.5 Let us now save this data into individual CSV files

In [54]:
df_train.to_csv('./data/online_price_train_df.csv', index=False)
train_X.to_csv('./data/online_price_train_X.csv', index=False)
train_y.to_csv('./data/online_price_train_y.csv', index=False)
df_test.to_csv('./data/online_price_test_df.csv', index=False)
test_X.to_csv('./data/online_price_test_X.csv', index=False)
test_y.to_csv('./data/online_price_test_y.csv', index=False)